In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Text Classification with Generative Models on Vertex AI

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/language/prompts/examples/text_classification.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> Run in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/language/prompts/examples/text_classification.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/blob/main/language/prompts/examples/text_classification.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Open in Vertex AI Workbench
    </a>
  </td>
</table>


## Overview

PaLM 2와 같은 생성 모델은 다양한 자연어 처리(NLP) 작업에 사용되는 강력한 언어 모델입니다. 그 중 하나가 텍스트 분류입니다. 이는 주어진 텍스트에 하나 이상의 범주를 할당하는 작업을 포함합니다. 전통적인 NLP 기술을 사용하여 텍스트 분류를 수행할 수 있지만 LLM은 프롬프트(도메인별 레이블이 지정된 데이터와 반대)를 제공하여 분류를 수행할 수 있으므로 텍스트 분류 솔루션을 구축하는 데 걸리는 시간을 단축할 수 있습니다. LLM 기반 분류 모델은 사용자 정의 모델 교육을 통해 많은 예제를 통해 추가로 조정할 수 있지만 이는 이 노트북의 범위를 벗어납니다.

이 노트북에서는 PaLM API의 프롬프트를 사용하여 텍스트 분류를 수행하는 방법을 살펴보겠습니다. [공식 문서](https://cloud.google.com/vertex-ai/docs/generative-ai/text/classification-prompts)에서 분류 프롬프트에 대해 자세히 알아보세요.

### 목적

노트북이 끝나면 대규모 언어 모델을 사용하여 다음을 포함한 다양한 분류 작업을 수행할 수 있게 됩니다.

* 제로샷 프롬프트 텍스트 분류
* 퓨샷 프롬프트 텍스트 분류
* 일반적인 작업:
     * 감성 분석
     * 주제 분류
     * 스팸 감지
     * 의도 인식
     * 언어 식별
     * 독성 감지
     * 감정 감지

## Getting Started

### Install Vertex AI SDK

In [ ]:
!pip install google-cloud-aiplatform --upgrade --user

**Colab only:** Uncomment the following cell to restart the kernel or use the button to restart the kernel. For Vertex AI Workbench you can restart the terminal using the button on top.

In [ ]:
# # Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

### Authenticating your notebook environment
* If you are using **Colab** to run this notebook, uncomment the cell below and continue.
* If you are using **Vertex AI Workbench**, check out the setup instructions [here](https://github.com/GoogleCloudPlatform/generative-ai/tree/main/setup-env).

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

### Import libraries

**Colab only:** Uncomment the following cell to initialize the Vertex AI SDK. For Vertex AI Workbench, you don't need to run this.  

In [ ]:
# import vertexai

# PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
# vertexai.init(project=PROJECT_ID, location="us-central1")

In [ ]:
import pandas as pd
from vertexai.language_models import TextGenerationModel

### Import models

In [ ]:
generation_model = TextGenerationModel.from_pretrained("text-bison@001")

## Text Classification

In the section below, you will explore zero-shot prompting, few-shot prompting, and some common types of text classification tasks.

### Zero-shot prompting

Zero-shot prompting is where you do not provide examples with labels, and rely on the LLM to make the classification on its own.

In [ ]:
prompt = """
다음을 분류하십시오.\n
텍스트: "오늘 공원에서 꼬리가 길고 눈이 큰 털복숭이 동물을 봤어요."
라벨: 개, 고양이
"""

print(
    generation_model.predict(
        prompt=prompt,
        max_output_tokens=256,
        temperature=0.1,
    ).text
)

### Few-shot prompting

With few-shot prompting, you provide examples to the PaLM model and expect it to predict classes based on the provided examples.

In [ ]:
prompt = """
특정 뉴스 헤드라인의 주제는 무엇입니까?
- 비즈니스
- 엔터테인먼트
- 건강
- 스포츠
- 기술

텍스트: Pixel 7 Pro 전문가 직접 검토.
답: 기술

문자: 담배를 끊으시겠어요?
답: 건강

텍스트: 버디인가, 보기인가? 언더파를 기록하기 위한 5가지 팁
답: 스포츠

텍스트: 지역 최저 임금 인상 완화로 인해 더 멀어질 전망
답: 비즈니스

텍스트: 영화 시사회를 위해 이탈리아 바리에 방금 도착한 사람이 누구인지 짐작할 수 없습니다.
답:
"""

print(
    generation_model.predict(
        prompt=prompt,
        max_output_tokens=256,
        temperature=0.1,
    ).text
)

### Other classification examples

아래에서 제로샷 프롬프트를 기반으로 하는 좀 더 일반적인 텍스트 분류 프롬프트를 살펴보세요. 또한 사용자 정의 텍스트 예제와 관련 출력 클래스를 제공하여 이들 중 일부를 몇 번의 프롬프트로 전환할 수도 있습니다.

#### Topic classification

In [ ]:
prompt = """
스포츠, 정치, 엔터테인먼트 등 미리 정의된 여러 주제 중 하나로 텍스트를 분류합니다.

텍스트: 바이든 대통령은 몇 가지 기회를 논의하기 위해 3월에 인도를 방문할 예정입니다.
분류:
"""

print(
    generation_model.predict(
        prompt=prompt,
        max_output_tokens=256,
        temperature=0.1,
    ).text
)

####  Spam detection

In [ ]:
prompt = """
이메일이 주어지면 스팸인지 스팸이 아닌지 분류하세요.

이메일: 안녕하세요, 사용자님,
       귀하는 복권 당첨자로 선정되었으며 최대 100만 달러까지 당첨될 수 있습니다.
       귀하의 은행 정보를 친절하게 공유해 주시면 거기서부터 진행할 수 있습니다.

       미국 공식 복권 부서
"""

print(
    generation_model.predict(
        prompt=prompt,
        max_output_tokens=256,
        temperature=0.1,
    ).text
)

#### Intent recognition

In [ ]:
prompt = """
사용자의 입력을 바탕으로 '정보 찾기', '예약하기', '주문하기' 등 의도를 분류합니다.
사용자 입력: 안녕하세요. 5월 1일 Juan에 2인용 테이블을 예약해 주실 수 있나요?
"""

print(
    generation_model.predict(
        prompt=prompt,
        max_output_tokens=256,
        temperature=0.1,
    ).text
)

#### Language identification

In [ ]:
prompt = """
텍스트가 주어지면 해당 텍스트가 작성된 언어를 분류하십시오. \n
텍스트: Selam nasıl gidiyor?
언어:
"""

print(
    generation_model.predict(
        prompt=prompt,
        max_output_tokens=256,
        temperature=0.1,
    ).text
)

#### Toxicity detection

In [ ]:
prompt = """
텍스트가 주어지면 긍정 또는 부정으로 분류하세요.
텍스트: 난 화창한 날을 좋아해
"""

print(
    generation_model.predict(
        prompt=prompt,
        max_output_tokens=256,
        temperature=0.1,
    ).text
)

#### Emotion detection

In [ ]:
prompt = """
텍스트가 주어지면 그것이 전달하는 감정(예: 행복, 분노)을 분류하세요.
문자: 어제 뉴스가 아직도 너무 기뻐요
"""

print(
    generation_model.predict(
        prompt=prompt,
        max_output_tokens=256,
        temperature=0.1,
    ).text
)

### Evaluation

텍스트 분류 작업의 출력을 평가할 수 있습니다. 어떻게 작동하는지 보여주기 위해 제품 리뷰와 실제 감정이 포함된 간단한 데이터 프레임을 만드는 것부터 시작하세요.

In [ ]:
review_data = {
    "review": [
        "나는 이 제품을 좋아합니다. 내가 찾고 있는 모든 것을 갖추고 있습니다!",
         "제가 드릴 수 있는 말은 이 제품을 구매하신 후 당신이 행복해질 것이라는 것뿐입니다.",
         "너무 비싸고 가격 대비 가치가 없습니다",
         "기분은 괜찮아요. 좋지도 나쁘지도 않아요."
    ],
    "sentiment_groundtruth": ["긍정", "긍정", "부정", "중립"],
}

review_data_df = pd.DataFrame(review_data)
review_data_df

이제 리뷰와 감정이 실측 라벨로 포함된 데이터가 있으므로 '적용' 기능을 사용하여 각 리뷰 행에 텍스트 생성 모델을 호출할 수 있습니다. 각 행은 'review' 열의 프롬프트를 사용하여 PaLM API를 사용하여 감정을 예측하고 결과를 'sentiment_prediction' 열에 저장합니다.

In [ ]:
def get_sentiment(row):
    prompt = f"""다음 리뷰의 감정을 "긍정", "중립", "부정"으로 분류합니다.
                리뷰: {row} \n
                감정:
              """
    response = generation_model.predict(prompt=prompt).text
    return response


review_data_df["sentiment_prediction"] = review_data_df["review"].apply(get_sentiment)
review_data_df

결국 sklearn에서 'classification_report' 함수를 호출하여 실제 감정 'sentiment_groundtruth' 및 예측된 감정 'sentiment_prediction'을 전달하여 정확도 및 기타 분류 측정항목을 측정할 수 있습니다.

In [ ]:
from sklearn.metrics import classification_report

print(
    classification_report(
        review_data_df["sentiment_groundtruth"], review_data_df["sentiment_prediction"]
    )
)